In [ ]:
pip install imgkit

In [2]:
import os
import pandas as pd
import dataframe_image as dfi

pd.set_option('display.max_rows',None, 'display.max_columns', None)

# CAPTURE UNPIVOT TABLE

In [9]:
df_quadran = pd.read_csv('../quadran/target.csv')
#target_pivot = df_quadran[['KABUPATEN/KOTA','Jumlah_DPT_2024','DPT_x_82%','TARGET_GP_2024','% TARGET_GP_2024']]
df_quadran.columns

Index(['PROPINSI', 'Kabupaten', 'KABUPATEN2', 'Kecamatan', 'Jumlah DPT 2024',
       ' DPT 2024 x 82% ', 'Target GP 2024', '% TARGET GP 2024',
       'Jumlah Kel/Desa', 'Jumah TPS', 'Prio', 'Kuadran'],
      dtype='object')

In [10]:
df_quadran.head()

,PROPINSI,Kabupaten,KABUPATEN2,Kecamatan,Jumlah DPT 2024,DPT 2024 x 82%,Target GP 2024,% TARGET GP 2024,Jumlah Kel/Desa,Jumah TPS,Prio,Kuadran
0,ACEH,ACEH SELATAN,ACEH SELATAN,BAKONGAN,3886,"3,186.5","1,578",49.51%,7.0,15.0,0,4
1,ACEH,ACEH SELATAN,ACEH SELATAN,KLUET UTARA,18230,"14,948.6","3,021",20.21%,21.0,71.0,0,4
2,ACEH,ACEH SELATAN,ACEH SELATAN,KLUET SELATAN,10686,"8,762.5","1,627",18.57%,17.0,44.0,0,4
3,ACEH,ACEH SELATAN,ACEH SELATAN,LABUHAN HAJI,9806,"8,040.9",956,11.89%,16.0,40.0,0,4
4,ACEH,ACEH SELATAN,ACEH SELATAN,MEUKEK,15479,"12,692.8","1,617",12.74%,23.0,64.0,0,4


In [ ]:
provinsi = [
    'BANTEN',
]
kabkot = df_quadran['KABUPATEN2'].unique().tolist()
kabkot

In [14]:
# def (jenis, jumlah_dpt):

#     print(jenis)
#     print(jumlah_dpt)
    # jumlah_dpt = jumlah_dpt.replace(',','').astype('int64')
    # dpt_82 = dpt_82.replace(',','').astype('int64')
    # target_gp = target_gp.replace(',','').astype('int64')

    # data_dict = {
    #     '' : [jenis],
    #     'Jumlah DPT 2024 tes' : jumlah_dpt.sum(),
    #     'DPT x 82% tes' : dpt_82.sum(),
    #     'TARGET GP 2024 tes' : target_gp.sum()}

def total(tipe, data):
    x = data.copy()
    total = x[['Jumlah Kel/Desa','Jumlah TPS','DPT 2024','DPT 2024 x 82%','Target GP 2024']].sum().to_frame().T
    total['Kecamatan'] = tipe

    df = pd.concat([x, total]) 
    return df

def move_column(data):
    first_column = data.pop('Jumlah DPT 2024')
    two_column = data.pop(' DPT 2024 x 82% ')
    thre_column = data.pop('Target GP 2024')
    tps = data.pop('Jumah TPS')
    kel_des = data.pop('Jumlah Kel/Desa')
    target_gp_per = data.pop('% TARGET GP 2024')
    
    data.insert(0, 'Jumlah Kel/Desa', kel_des)
    data.insert(1, 'Jumlah TPS', tps)
    data.insert(2, 'DPT 2024', first_column)
    data.insert(3, 'DPT 2024 x 82%', two_column)
    data.insert(4, 'Target GP 2024', thre_column)
    data.insert(5, '% TARGET GP 2024', target_gp_per)

    data['Kuadran'].replace('.0','').astype('int64')
    
    return data

def format_color_groups(df):
    colors = ['','#FFCECD', '#FFF4D9','','#EBF1F1']
    x = df.copy()
    factors = list(x['Kuadran'].unique())

    for factor in factors:
        style = f'background-color: {colors[factor]}; color:black;'
        x.loc[x['Kuadran'] == factor, :] = style
    return x


In [ ]:
styled_df = move_column_prio.style.apply(format_color_groups, axis=None)
styled_df

In [7]:
df_quadran.columns

Index(['kode_pro_kemendagri_2022', 'kode_kab_kemendagri_2022', 'nama_provinsi',
       'Kabupaten/Kota', 'PKB', 'Gerindra', 'PDIP', 'Golkar', 'Nasdem',
       'Garuda', 'Berkarya', 'PKS', 'Perindo', 'PPP', 'PSI', 'PAN', 'Hanura',
       'Demokrat', 'PA', 'Partai SIRA', 'PD Aceh', 'PNA', 'PBB', 'PKPI'],
      dtype='object')

# TINGKAT KABUPATEN

In [ ]:
data_null = []

for prop in range(0, len(provinsi)):
    for kab in range(0, len(kabkot)):

        try:
            prio = df_quadran[(df_quadran['PROPINSI'] == provinsi[prop]) & (df_quadran['KABUPATEN2'] == kabkot[kab]) & (df_quadran['Prio'] == 1)].pivot_table(index='Kecamatan', values=['Jumlah DPT 2024',' DPT 2024 x 82% ','Target GP 2024','Jumah TPS','Jumlah Kel/Desa','% TARGET GP 2024','Kuadran'], aggfunc='sum')
            non_prio = df_quadran[(df_quadran['PROPINSI'] == provinsi[prop]) & (df_quadran['KABUPATEN2'] == kabkot[kab]) & (df_quadran['Prio'] == 0)].pivot_table(index='Kecamatan', values=['Jumlah DPT 2024',' DPT 2024 x 82% ','Target GP 2024','Jumah TPS','Jumlah Kel/Desa','% TARGET GP 2024','Kuadran'], aggfunc='sum')
            prio_non_prio = df_quadran[(df_quadran['PROPINSI'] == provinsi[prop]) & (df_quadran['KABUPATEN2'] == kabkot[kab])].pivot_table(index='Kecamatan', values=['Jumlah DPT 2024',' DPT 2024 x 82% ','Target GP 2024','Jumah TPS','Jumlah Kel/Desa','% TARGET GP 2024'], aggfunc='sum')

            if prio.empty or non_prio.empty:
                pass
            else:
                move_column_prio = move_column(prio)
                move_column_non_prio = move_column(non_prio)
                
                move_column_prio.reset_index(inplace=True)
                move_column_non_prio.reset_index(inplace=True)

                move_column_prio.index += 1
                move_column_non_prio.index += 1

                #styled_prio = move_column_prio.style.apply(format_color_groups, axis=None)
                print(prio_non_prio)
                #styled_non_prio = move_column_non_prio.style.apply(format_color_groups, axis=None)
                
                # update_prio = total('Total Prioritas', move_column_prio)
                # update_non_prio = total('Total Non Prioritas', move_column_non_prio)
                
                # grand_total = prio_non_prio[['Jumlah Kel/Desa','Jumlah TPS','DPT 2024','DPT 2024 x 82%','Target GP 2024']].sum().to_frame().T
                # grand_total['Kecamatan'] = tipe
                # print(update_all)

                #result_prio = color_prio(update_prio)
                # result_non_prio = color_non_prio(update_non_prio)
                #result_total_prio = color_prio(total_prio)

                # outdir = f'{provinsi[prop]}/{kabkot[kab]}'
                # if not os.path.exists(outdir):
                #     os.mkdir(outdir)

                # dfi.export(styled_prio, f'../quadran/result/{provinsi[prop]}-{kabkot[kab]}-prioritas.jpg')
                # dfi.export(result_total_prio, f'result/{provinsi[prop]}-{kabkot[kab]}-grand_total_prioritas.jpg')
                # dfi.export(result_non_prio, f'result/{provinsi[prop]}-{kabkot[kab]}-non_prioritas.jpg')

        except KeyError as e:
            print(f"Error key not found {e}")
            break
        
    #print("Gagal Running Mase")
    # unpivot.insert(0, 'Jumlah DPT 2024', first_column)
    # unpivot.reset_index(inplace=True)

    #dfi.export(unpivot, f'result/{provinsi[prop]}.jpg')

In [341]:
import pandas as pd

# Dataframe awal
data = {
    'Kecamatan': ['LIMUN', 'PAUH', 'BATANG ASAI'],
    'Jumlah Kel/Desa': [16, 14, 23],
    'Jumah TPS': [61, 72, 67],
    'DPT 2024': [13.325, 17.699, 13.985]
}

df = pd.DataFrame(data)

# Menghitung total
total_nonprioritas = df[['Jumlah Kel/Desa', 'Jumah TPS', 'DPT 2024']].sum().to_frame().T
total_nonprioritas['Kecamatan'] = 'Total Nonprioritas'

# Menggabungkan dataframe
df = pd.concat([df, total_nonprioritas], ignore_index=True)

outdir = 'dir'
if not os.path.exists(outdir):
    os.mkdir(outdir)

df.to_csv(f'{outdir}/asw.csv')


# CAPTURE TABLE WITH COLOR

In [313]:
file_dpr_ri = '../request_mas_zima/capture_prioritas/0112022_2019_pers_suara_dpr_ri_kab.csv'
file_pilpres = '../request_mas_zima/capture_prioritas/0111012_2019_suara_pilpres_kab.csv'

pileg = pd.read_csv(file_dpr_ri)
pilpres = pd.read_csv(file_pilpres)

pilpres.head()

,kode_pro_kemendagri_2022,kode_kab_kemendagri_2022,nama_pro,nama_kab,JW-MA,PS-SU,JW-MA(%),PS-SU(%)
0,11,1101,ACEH,ACEH SELATAN,12568,120516,0.094437,0.905563
1,11,1102,ACEH,ACEH TENGGARA,34039,91689,0.270735,0.729265
2,11,1103,ACEH,ACEH TIMUR,21413,178143,0.107303,0.892697
3,11,1104,ACEH,ACEH TENGAH,47464,73503,0.392371,0.607629
4,11,1105,ACEH,ACEH BARAT,15823,97388,0.139766,0.860234


In [314]:
columns_pileg = pileg.columns.to_list()
columns_pilpres = pilpres.columns.to_list()
provinsi = pileg['nama_provinsi'].unique().tolist()

#provinsi = ['BANTEN']
columns_pilpres

['kode_pro_kemendagri_2022',
 'kode_kab_kemendagri_2022',
 'nama_pro',
 'nama_kab',
 'JW-MA',
 'PS-SU',
 'JW-MA(%)',
 'PS-SU(%)']

In [315]:
pileg['Max Value'] = pileg[columns_pileg[4:]].max(axis=1)
pileg['pemenang'] = pileg[columns_pileg[4:]].idxmax(axis=1)

pilpres['Max Value'] = pilpres[columns_pilpres[6:]].max(axis=1)
pilpres['pemenang'] = pilpres[columns_pilpres[6:]].idxmax(axis=1)

columns_pileg = pileg.columns.to_list()

pileg.head(5)

,kode_pro_kemendagri_2022,kode_kab_kemendagri_2022,nama_provinsi,Kabupaten/Kota,PKB,Gerindra,PDIP,Golkar,Nasdem,Garuda,Berkarya,PKS,Perindo,PPP,PSI,PAN,Hanura,Demokrat,PA,Partai SIRA,PD Aceh,PNA,PBB,PKPI,Max Value,pemenang
0,11,1101,ACEH,ACEH SELATAN,0.074087,0.040443,0.008268,0.158766,0.065081,0.004417,0.021264,0.063004,0.006724,0.161451,0.011978,0.156894,0.007964,0.186204,0,0,0,0,0.017487,0.015969,0.186204,Demokrat
1,11,1102,ACEH,ACEH TENGGARA,0.128523,0.020689,0.010146,0.577875,0.008933,0.003707,0.008966,0.015083,0.003079,0.004210,0.002097,0.172195,0.014447,0.025089,0,0,0,0,0.004268,0.000693,0.577875,Golkar
2,11,1103,ACEH,ACEH TIMUR,0.058135,0.248730,0.031508,0.079574,0.092293,0.013771,0.027757,0.070617,0.010978,0.099801,0.019814,0.034664,0.008604,0.171272,0,0,0,0,0.030643,0.001840,0.248730,Gerindra
3,11,1104,ACEH,ACEH TENGAH,0.038254,0.264274,0.296404,0.061277,0.038489,0.004971,0.026670,0.055480,0.008273,0.074158,0.010260,0.031749,0.018251,0.058092,0,0,0,0,0.012047,0.001352,0.296404,PDIP
4,11,1105,ACEH,ACEH BARAT,0.032084,0.089928,0.030201,0.121137,0.074394,0.004832,0.042738,0.095292,0.007572,0.071274,0.010539,0.163333,0.007610,0.233437,0,0,0,0,0.012252,0.003377,0.233437,Demokrat


In [316]:
warna_partai = {
    'PDIP': 'background-color: #db2016; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Kabupaten/Kota': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'Golkar': 'background-color: #ffff00 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Gerindra': 'background-color: #b79164 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Nasdem': 'background-color: #99d9ea ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PKB': 'background-color: #008000 ; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Demokrat': 'background-color: #2643a3 ; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PAN': 'background-color: #0033ff ; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PKS': 'background-color: #000000 ; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PPP': 'background-color: #00b300 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Perindo': 'background-color: #7583a9 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Berkarya': 'background-color: #eeee22 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Hanura': 'background-color: #ed9a11 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PSI': 'background-color: #ff00ff ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PBB': 'background-color: #00ff00 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Garuda': 'background-color: #e6b333 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PKPI': 'background-color: #ff0000 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PD Aceh': 'background-color: #118dff ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PA': 'background-color: #fa8072 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Partai SIRA': 'background-color: #00beff ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PNA': 'background-color: #fbceb1 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'pemenang':''}

def select_col(source):
    
    filter = source.columns.to_list()
    filter.remove('Kabupaten/Kota')
    filter.remove('Max Value')

    df1 =  pd.DataFrame(index=source.index, columns=source.columns)

    for col in filter:
        compare = source[col].eq(source['Max Value'])
        df1.loc[compare, col] = warna_partai[col]

    return df1

def table_style(source):
    filter = source.columns.to_list()
    filter.remove('Max Value')

    for col in filter:

        font_css = '"Montserrat", sans-serif;'
        header_styles = {
            col : [{'selector': 'th',  'props': warna_partai[col]}],
        }
        
        source = source.set_table_styles(header_styles, overwrite=False)
        format_dict = {col: '{:.1%}' for col in filter if col != 'Kabupaten/Kota'}
        convert = source.format(format_dict)
       
        convert.set_properties(**{'text-align':'center','font-family': font_css})

    return convert

def move_column_pilpres(data):
    first_column = data.pop('PS-SU')
    two_column = data.pop('PS-SU(%)')	
    
    data.insert(2, 'PS-SU', first_column)
    data.insert(4, 'PS-SU(%)', two_column)

    return data

def classify_pilpres(jwma):
    if 0 <= jwma < 0.15:
        return 'A'
    elif 0.15 <= jwma < 0.30:
        return 'B'
    elif 0.30 <= jwma < 0.45:
        return 'C'
    elif 0.45 <= jwma < 0.50:
        return 'D'
    elif 0.50 <= jwma < 0.55:
        return 'F'
    elif 0.55 <= jwma < 0.70:
        return 'G'
    elif 0.70 <= jwma < 0.85:
        return 'H'
    else:
        return 'I'

In [371]:
def pilpres_col(source):
    background = {
        'A' : 'background-color: #eaa00a; color: white; font-weight: bold;',
        'B' : 'background-color: #ffba14; color: white; font-weight: bold;',
        'C' : 'background-color: #fed12b; color: white; font-weight: bold;', 
        'D' : 'background-color: #f9d97d; color: white; font-weight: bold;', 
        'F' : 'background-color: #ff8080; color: white; font-weight: bold;', 
        'G' : 'background-color: #ff6666; color: white; font-weight: bold;', 
        'H' : 'background-color: #ff5550; color: white; font-weight: bold;',
        'I' : 'background-color: #ff2929; color: white; font-weight: bold;'}
    
    column = ['pilpres_jwma','pilpres_klas']
    df1 = pd.DataFrame(index=source.index, columns=source.columns)

    for x in column:
        mask = source['pemenang'] == 'PS-SU(%)'
        df1.loc[mask, source[x]] = background[source[x]]
    
    return df1

    # for x in source[mask]:
    #     print(x)
        # if mask:
        #     df1.loc[mask, 'PS-SU(%)'] = background[source['pilpres_jwma']]
        # else:
        #     df1.loc[mask, 'JW-MA(%)'] = background[source['pilpres_klas']]
    
    #return df1        

In [ ]:
ddd = move_pilpres.style.apply(pilpres_col, axis=None)
ddd

In [ ]:
col = ['pilpres_jwma','pilpres_klas']

background = {
    'A' : 'background-color: #eaa00a; color: white; font-weight: bold;',
    'B' : 'background-color: #ffba14; color: white; font-weight: bold;',
    'C' : 'background-color: #fed12b; color: white; font-weight: bold;', 
    'D' : 'background-color: #f9d97d; color: white; font-weight: bold;', 
    'F' : 'background-color: #ff8080; color: white; font-weight: bold;', 
    'G' : 'background-color: #ff6666; color: white; font-weight: bold;', 
    'H' : 'background-color: #ff5550; color: white; font-weight: bold;',
    'I' : 'background-color: #ff2929; color: white; font-weight: bold;'}

mask = move_pilpres['pemenang'] == 'JW-MA(%)'
tes = move_pilpres[mask]
for x in tes:
    print(move_pilpres[x])
    # mask = move_pilpres['pemenang'] == 'JW-MA(%)'
    # asd = move_pilpres[x].unique()
    # move_pilpres.loc[mask, 'JW-MA'] = background[asd]
    #print(move_pilpres[x].unique())

In [ ]:
pilpres['pilpres_klas'] = pilpres['Max Value'].apply(classify_pilpres)
pilpres['pilpres_jwma'] = pilpres['JW-MA(%)'].apply(classify_pilpres)
columns_pilpres = pilpres.columns.to_list()
pilpres[(pilpres['pemenang'] == 'PS-SU(%)')]

In [318]:
styles = {
    'Kabupaten/Kota' : [{'selector': 'td', 'props': [('text-align', 'left')]}]
}

for pro in provinsi:
    unpivot_pileg = pileg[(pileg['nama_provinsi'] == pro)].pivot_table(index='Kabupaten/Kota', values=columns_pileg[4:], aggfunc='sum')
    unpivot_pilpres = pilpres[(pilpres['nama_pro'] == pro)].pivot_table(index='nama_kab', values=columns_pilpres[4:], aggfunc='sum')

    if unpivot_pileg.empty or unpivot_pilpres.empty:
        pass
    else:
        #pilpres
        unpivot_pilpres.reset_index(inplace=True)
        move_pilpres = move_column_pilpres(unpivot_pilpres)

        # coloring_pileg = filter_pileg.style.apply(select_col, axis=None)
        # coloring_pileg = coloring_pileg.hide(['Max Value'], axis=1)
        # coloring_pileg = table_style(coloring_pileg)

        #pileg
        unpivot_pileg.reset_index(inplace=True)
        pemenang_pileg = unpivot_pileg['pemenang'].unique().tolist()
        pemenang_pileg.insert(0, 'Kabupaten/Kota')
        filter_pileg = unpivot_pileg[pemenang_pileg]
        filter_pileg['Max Value'] = filter_pileg[pemenang_pileg[1:]].max(axis=1)

        coloring_pileg = filter_pileg.style.apply(select_col, axis=None)
        coloring_pileg = coloring_pileg.set_table_styles(styles, overwrite=False)
        coloring_pileg = coloring_pileg.hide(['Max Value'], axis=1)
        coloring_pileg = table_style(coloring_pileg)
        
        #dfi.export(coloring_pileg, f'../request_mas_zima/capture_prioritas/{pro}.jpg')

C:\Users\User\AppData\Local\Temp\ipykernel_17524\310713769.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_pileg['Max Value'] = filter_pileg[pemenang_pileg[1:]].max(axis=1)
C:\Users\User\AppData\Local\Temp\ipykernel_17524\310713769.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_pileg['Max Value'] = filter_pileg[pemenang_pileg[1:]].max(axis=1)
C:\Users\User\AppData\Local\Temp\ipykernel_17524\310713769.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [282]:
coloring_pileg

,Kabupaten/Kota,Gerindra,PDIP,PKS
0,KOTA CILEGON,18.0%,11.3%,13.8%
1,KOTA SERANG,16.6%,10.4%,11.5%
2,KOTA TANGERANG,14.5%,23.1%,12.9%
3,KOTA TANGERANG SELATAN,12.7%,24.2%,15.1%
4,LEBAK,21.0%,12.5%,8.4%
5,PANDEGLANG,12.8%,8.8%,14.6%
6,SERANG,17.2%,8.3%,10.5%
7,TANGERANG,12.8%,17.1%,9.7%


In [ ]:
result = partai.style.apply(select_col, axis=None)
result = result.hide(['Max Value'], axis=1)
result

# #add color per column
header_styles = {
    'nama_provinsi': [{'selector': 'th','props': [('background-color', f'{pdip}'),('color','white'),('font-family', 'Montserrat'),('text-align','center')]}],
    'Kabupaten/Kota': [{'selector': 'th','props': [('background-color', f'{pdip}'),('color','white'),('font-family', 'Montserrat'),('text-align','center')]}],
    'Gerindra': [{'selector': 'th','props': [('background-color', f'{gerindra}'),('color','black'),('font-family', 'Montserrat'),('text-align','center')]}],
    'Golkar': [{'selector': 'th','props': [('background-color',f'{golkar}'),('color','black'),('font-family', 'Montserrat'),('text-align','center')]}],
    'PDIP': [{'selector': 'th','props': [('background-color',f'{pdip}'),('color','white'),('font-family', 'Montserrat'),('text-align','center')]}],
    'PKS': [{'selector': 'th','props': f'background-color:{pks}; color:white; font-family: Montserrat; text-align: center;'}]
}

result.set_table_styles(header_styles, overwrite=False)
convert = result.format({'Gerindra': '{:.1%}','PDIP': '{:.1%}','Golkar': '{:.1%}','PKS': '{:.1%}'})

font_css = "font-family: 'Montserrat';"
convert.set_properties(**{'text-align': 'center', 'font-family': font_css})

In [24]:
dfi.export(convert, '../upload_db/data_pemilu_2019/dpr_ri/dpr ri partai.jpg', max_rows=-1)